In [ ]:
import numpy as np
import matplotlib as plt
%config InlineBackend.figure_format = 'retina'

In [ ]:
import re

In [ ]:
max_length_repeat = 50
min_length_repeat = 5
hist_bins = np.append(np.arange(min_length_repeat,max_length_repeat+2)-0.5,np.inf)
split_cond = 'C|G|T'
gene_list = []
len_list = []
mean_length = []

with open('../gene_search/human_all_genes1.fasta','r') as f:
    c = f.readlines()
    header_lines = [line_num for line_num in range(len(c)) if c[line_num][0] == '>']

    n_genes = len(header_lines)
    n_bins = len(hist_bins)-1
    hist = np.zeros((n_genes,n_bins))
    
    print(n_genes)
#     for jj in range(5):
    for jj in range(n_genes):
        gene_beg = header_lines[jj]
        if jj<len(header_lines)-1:
            gene_end = header_lines[jj+1]
        else:
            gene_end = len(c)
        seq = ''.join([line[0:-1] for line in c[gene_beg+1:gene_end]])
        
        header = c[gene_beg][1:-1]
        header_div = [n for n in range(len(header)) if header[n] == '|']

        gene_start = header[(header_div[1]+1):(header_div[2])]
        gene_end = header[(header_div[2]+1):(header_div[3])]
        gene_len = int(gene_end)-int(gene_start)+1
        
        spl_str_len = [len(i) for i in [i for i in re.split(split_cond,seq) if i]]
        if max(spl_str_len) >= min_length_repeat:
            hist[jj,:] = np.histogram(spl_str_len,hist_bins)[0]

        if np.mod(jj,100) == 0:
            print(str(round(jj/n_genes*100,3))+'%',end='\t')
        
        len_list.append(gene_len)
        gene_name = header[(header_div[5]+1):]
        gene_list.append(gene_name)

In [ ]:
hist_int = hist.astype('int32')

In [ ]:
with open('../gene_search/gg_200525_genome_polyA_3','w') as file:
    for i in range(n_genes):
        file.write(gene_list[i]+' '+str(len_list[i])+' '+np.array2string(
            hist_int[i,:],max_line_width=np.inf,separator=' ')[1:-1]+'\n')

In [ ]:
hist_cum = np.fliplr(np.cumsum(np.fliplr(hist_int),1))

In [ ]:
with open('../gene_search/gg_200525_genome_polyA_cum_3','w') as file:
    for i in range(n_genes):
        file.write(gene_list[i]+' '+str(len_list[i])+' '+np.array2string(
            hist_cum[i,:],max_line_width=np.inf,separator=' ')[1:-1]+'\n')